In [1]:
%matplotlib inline

from scipy.odr import *
from scipy.stats import *
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import ast
from multiprocessing import Pool

import scipy

from IPython import display
from matplotlib.patches import Rectangle

from sklearn.metrics import mean_squared_error
import json

import scipy.stats as st
from sklearn.metrics import r2_score


from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

import copy

from sklearn.model_selection import LeaveOneOut, LeavePOut

from multiprocessing import Pool
import cv2

In [5]:
dfAll = pd.read_pickle("PklData/df_blobs.pkl")
dfAll.head()

,userID,Timestamp,Current_Task,Task_amount,TaskID,VersionID,RepetitionID,Actual_Data,Is_Pause,Image,BlobCount,Blobs
10000,3,1545216480595,1,680,10,2,0,True,False,"[0, 227, 191, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1,"[[[0, 227, 191, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0..."
10004,3,1545216480764,1,680,10,2,0,True,False,"[0, 227, 191, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,..."
10005,3,1545216480809,1,680,10,2,0,True,False,"[0, 227, 191, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,..."
10006,3,1545216480845,1,680,10,2,0,True,False,"[0, 227, 191, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,..."
10007,3,1545216480886,1,680,10,2,0,True,False,"[0, 227, 191, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,..."


In [6]:
dfAll.iloc[dfAll.groupby(['userID', 'TaskID', 'VersionID'])['RepetitionID'].agg(pd.Series.idxmax)]

IndexError: positional indexers are out-of-bounds